In [1]:
import sys

import datetime
import pickle

import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats

import glob

from scipy.stats import norm
from sklearn.neighbors import KernelDensity



import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly

import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [12]:
cont = 0
for filename in glob.glob('/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/*.csv'):
    print (filename)
    df_census_city =pd.read_csv(filename)
    cont +=1
df_census_city
print (cont)

/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AZ_Phoenix.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/ID_Boise.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/LA_Shreveport.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AR_Pine_Bluff.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/HI_Kailua.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/WI_Green_Bay.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MD_Baltimore.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MD_Frederick.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MN_Rochester.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/TN_Nashville.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/RI_Warwick.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AK_Anchorage.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/SD_Sioux_Falls.csv
/home/

/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MA_Boston.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/OR_Eugene.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AK_Fairbanks.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/KY_Lexington.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MA_Springfield.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/TX_San_Antonio.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MT_Billings.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/ID_Nampa.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IL_Aurora.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AR_Fayetteville.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AL_Birmingham.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/WA_Spokane.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/NM_Las_Cruces.csv
/home/

In [2]:
for filename in glob.glob('/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IL*.csv'):
    print (filename)
    df_census_city =pd.read_csv(filename)
    
    df_census_city['Year'] = df_census_city['Year'].astype(int)
    
    data = []
    trace1 = go.Scatter(
                x = df_census_city['Year'],
                y = df_census_city['TotalPop'],
               # name = state,

            )
    data.append(trace1)


    # Edit the layout
    layout = dict(
    #     title = 'Average High and Low Temperatures in New York',
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Total population'),
                  )

    fig = dict(data=data, layout=layout)


    plotly.offline.iplot(fig)


# offline.plot(fig, auto_open=True, image = 'png', image_filename="time_evol_population" ,image_width=2000, image_height=1000, 
#               filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/time_evol_population.html', validate=True)




/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IL_Chicago.csv


/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IL_Rockford.csv


/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IL_Aurora.csv


In [3]:
df_census_city

,AsianPop,BlackPop,City,HispanicPop,NativePop,State,TotalPop,WhitePop,Year
0,NaN,657,Aurora,0,NaN,IL,16986,16329,1960
1,NaN,1748,Aurora,0,NaN,IL,23410,21492,1970
2,NaN,3264,Aurora,5942,NaN,IL,30203,23886,1980
3,504.0,4327,Aurora,9293,80.0,IL,36395,26260,1990
4,1482.0,4910,Aurora,19599,198.0,IL,48403,31402,2000
5,3512.0,6521,Aurora,28644,329.0,IL,60606,34405,2010


In [55]:
df2

,AsianPop,BlackPop,City,HispanicPop,NativePop,State,TotalPop,WhitePop,Year
0,nan,nan,Aurora,nan,nan,IL,nan,nan,2017


In [32]:
###### dont need to run again! already added the files###
######################


for filename in glob.glob('/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/*.csv'):
    print (filename)
    df_census_city = pd.read_csv(filename)
    
    
    name = filename.split('/')[-1]
    path = filename.replace(name, '')
    #print (path, name)

    
    
    state = df_census_city.loc[0]['State']  # i take the value from the first row (they are all the same so it doesnt matter)
    city = df_census_city.loc[0]['City']


    df_census_city['Year'] = df_census_city['Year'].astype(int)
    frames = [df_census_city]

    
    # i create new rows for interpolating, populating the year, city and state fields, and nan in the rest:
    year = 1960
    while year <= 2010:        
        if int(year) not in df_census_city.Year.unique():

            df2 = pd.DataFrame(columns=['AsianPop','BlackPop','City','HispanicPop','NativePop','State','TotalPop','WhitePop','Year'])
            df2.loc[0] = np.array([np.nan, np.nan, city, np.nan, np.nan, state, np.nan, np.nan, year])
            frames.append(df2)
        year +=1
        
        

    df_total = pd.concat(frames,ignore_index=True)
    df_total['Year'] = df_total['Year'].astype(int)
    
    df_total.sort_values(by=['Year'], inplace = True)
    
    
    df_total.reset_index(drop=True, inplace=True)
    
    df_total.to_csv( path+'interpolations/'+'interpolation_'+name, sep=',', na_rep='NA', index=False)
    print ("written:"+ path+'interpolations/'+'interpolation_'+name)


/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AZ_Phoenix.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_AZ_Phoenix.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/ID_Boise.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_ID_Boise.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/LA_Shreveport.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_LA_Shreveport.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AR_Pine_Bluff.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_AR_Pine_Bluff.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/HI_Kailua.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_HI_Kailua.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census

written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_WV_Huntington.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/NH_Manchester.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_NH_Manchester.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/RI_Cranston.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_RI_Cranston.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IN_Indianapolis.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_IN_Indianapolis.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MO_Springfield.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_MO_Springfield.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/NH_Nashua.csv
written:/home/staff/julia/at_Northwes

written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_CO_Aurora.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/SC_Charleston.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_SC_Charleston.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/WA_Seattle.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_WA_Seattle.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MN_Minneapolis.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_MN_Minneapolis.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IA_Des_Moines.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_IA_Des_Moines.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/NJ_Jersey_City.csv
written:/home/staff/julia/at_Northwestern/

written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_MN_Saint_Paul.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/CA_San_Diego.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_CA_San_Diego.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/OK_Tulsa.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_OK_Tulsa.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/TN_Memphis.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_TN_Memphis.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/WI_Milwaukee.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/interpolations/interpolation_WI_Milwaukee.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/WY_Casper.csv
written:/home/staff/julia/at_Northwestern/US_Crime/Data/Ada

In [10]:
list_files = []
for filename in glob.glob('/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/*.csv'):
    print (filename)
    list_files.append(filename)
    

/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AZ_Phoenix.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/ID_Boise.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/LA_Shreveport.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AR_Pine_Bluff.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/HI_Kailua.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/WI_Green_Bay.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MD_Baltimore.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MD_Frederick.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/MN_Rochester.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/TN_Nashville.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/RI_Warwick.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/AK_Anchorage.csv
/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/SD_Sioux_Falls.csv
/home/

In [33]:
def get_x_y_fit (df, col_x, col_y, pol_grade):
    
    x = np.array(df[col_x].tolist())
    y = np.array(df[col_y].tolist())
    
    z = np.polyfit(x, y, pol_grade)  ## fit to a n-grade polinomial. It returns array with Polynomial coefficients, highest power first


    # for plotting the fit
    max_x = max(x)
    min_x = min(x)
    
    x_fit = np.arange(min(x),max_x,1)  ### lin space for the coordenate x

    
    if pol_grade == 1:
        y_fit =  x_fit*z[0] + z[1]
    elif pol_grade == 2:
        y_fit =  x_fit*x_fit*z[0] + x_fit*z[1] + z[2]
    elif pol_grade == 3:
        y_fit = x_fit*x_fit*x_fit*z[0] + x_fit*x_fit*z[1] + x_fit*z[2] + z[3]
    elif pol_grade == 4:
        y_fit =  x_fit*x_fit*x_fit*x_fit*z[0] + x_fit*x_fit*x_fit*z[1] + x_fit*x_fit*z[2] + x_fit*z[3] + z[4]
    elif pol_grade == 5:
        y_fit =   x_fit*x_fit*x_fit*x_fit*x_fit*z[0] +x_fit*x_fit*x_fit*x_fit*z[1] +\
        x_fit*x_fit*x_fit*z[2] + x_fit*x_fit*z[3] + x_fit*z[4] + z[5]


    
    
    
    
    return x_fit, y_fit

In [30]:
df_census_city = pd.read_csv('/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/WV_Charleston.csv')

print (filename)



city = filename.split('/')[-1].replace('.csv','')

x = np.array(df_census_city[col_x].tolist())
y = np.array(df_census_city[col_y].tolist())

dict_year_real_value = dict(zip(df_census_city[col_x], df_census_city[col_y]))

x_fit, y_fit = get_x_y_fit(df_census_city, col_x, col_y, pol_grade)

dict_year_fit_value = dict(zip(x_fit, y_fit))


dict_year_value

/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IL_Chicago.csv


{1960: 912195,
 1970: 988936,
 1980: 1062629,
 1990: 972776,
 2000: 973413,
 2010: 921511}

In [223]:
for llave in dict_year_real_value:
    dict_year_fit_value[llave] = dict_year_real_value[llave]
    
dict_year_fit_value

{1960: 22011,
 1961: 21657.867973566055,
 1962: 21697.570474386215,
 1963: 21710.625592470169,
 1964: 21698.14666390419,
 1965: 21661.247021913528,
 1966: 21601.039997339249,
 1967: 21518.638926267624,
 1968: 21415.157140016556,
 1969: 21291.707973718643,
 1970: 19714,
 1971: 20989.360831022263,
 1972: 20812.689521074295,
 1973: 20620.504163980484,
 1974: 20413.91809296608,
 1975: 20194.044640302658,
 1976: 19961.997140169144,
 1977: 19718.888926267624,
 1978: 19465.833330392838,
 1979: 19203.943687677383,
 1980: 20470,
 1981: 18658.115592241287,
 1982: 18376.403806447983,
 1983: 18090.311307191849,
 1984: 17800.951425790787,
 1985: 17509.437497377396,
 1986: 17216.882854700089,
 1987: 16924.400830984116,
 1988: 16633.104758501053,
 1989: 16344.107973337173,
 1990: 15858,
 1991: 15777.465592622757,
 1992: 15502.04666352272,
 1993: 15233.380355119705,
 1994: 14972.579997301102,
 1995: 14720.758926153183,
 1996: 14479.030473947525,
 1997: 14248.507973909378,
 1998: 14030.304759263992,
 1

In [224]:
dict_year_real_value

{1960: 22011, 1970: 19714, 1980: 20470, 1990: 15858, 2000: 13068, 2010: 13025}

In [60]:
#### creating the interpolation files for each city, as well as plotting them:

pol_grade = 3   

col_x = 'Year'
col_y = 'TotalPop'   #BlackPop'    #TotalPop'


flag_plot = 'yes'
    

    
# # for filename in list_files:



#filename = '/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/IL_Chicago.csv'


cont = 1
for filename in glob.glob('/home/staff/julia/at_Northwestern/US_Crime/Data/Adam/census/NY*.csv'):
#     print (filename)


    print (cont)

    df_census_city = pd.read_csv(filename)

    name = filename.split('/')[-1]
    path = filename.replace(name, '')
    city = filename.split('/')[-1].replace('.csv','')
 



    ### plot the data vs the polynomial fit:   

    x = np.array(df_census_city[col_x].tolist())
    y = np.array(df_census_city[col_y].tolist())

    dict_year_value = dict(zip(df_census_city[col_x], df_census_city[col_y]))

    x_fit, y_fit = get_x_y_fit(df_census_city, col_x, col_y, pol_grade)



    trace1 = go.Scatter(
        x = x,
        y = y,
        name = 'data',
        mode = 'markers',  # 'markers+lines'
        marker=dict(
            color = '#006d2c', 
            size = 12)
    )

    trace2 = go.Scatter(
        x = x_fit,
        y = y_fit,
        name='fit',
        mode = 'lines',
        line = dict(
            color = ('#66c2a4'),
            width = 4,
            dash = 'dash')
    )

    data = [ trace1, trace2]

    # Edit the layout
    layout = dict(title = city,
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = col_y),
                  )

    fig = dict(data=data, layout=layout)
    # py.iplot(fig, filename='styled-line')

    
    if flag_plot == 'yes':
    
        offline.plot(fig, auto_open=True, image = 'png', image_filename="time_evol_population_"+city ,image_width=2000, image_height=1000, 
                   filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/time_evol_population_'+city+'.html', validate=True)









    ########### now i populate the interpolation files (preserving data for the years i do have):

    dict_year_real_value = dict(zip(df_census_city[col_x], df_census_city[col_y]))   
    dict_year_fit_value = dict(zip(x_fit, y_fit))



    for llave in sorted(dict_year_real_value):  # replace fit values for real ones in the census years specifically
        dict_year_fit_value[llave] = dict_year_real_value[llave]

    list_values = []    
    for llave in sorted(dict_year_fit_value): 
        list_values.append(int(dict_year_fit_value[llave]))


    interpolation_file = path+'interpolations/'+'interpolation_'+name
    df_interpolation = pd.read_csv(interpolation_file)


    new_col =  col_y +"_"+'interpolation'
    df_interpolation[new_col] = list_values 



    ##### i print out the interpolation file
    df_interpolation.to_csv( interpolation_file, sep=',', na_rep='NA', index=False)
   # print ("written:", interpolation_file)
    
    cont +=1

1
2
3


In [41]:
df_interpolation


,AsianPop,BlackPop,City,HispanicPop,NativePop,State,TotalPop,WhitePop,Year
0,NaN,238620.0,Chicago,0.0,NaN,IL,912195.0,673575.0,1960
1,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1961
2,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1962
3,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1963
4,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1964
5,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1965
6,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1966
7,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1967
8,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1968
9,NaN,NaN,Chicago,NaN,NaN,IL,NaN,NaN,1969
